<a href="https://colab.research.google.com/github/nunsongi/quito-risk-mapping-python/blob/main/02_lluvias_quito_inamhi_etl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧼 ETL de datos de precipitación (INAMHI) para Quito

Este notebook realiza la limpieza y transformación del dataset de lluvias y exporta un CSV listo para usar en el notebook principal de análisis de riesgo por lluvias en Quito.

---

## Contenido

1. [Contexto y objetivo del ETL](#1-contexto-y-objetivo-del-etl)  
2. [Configuración del Entorno y Librerías](#2-configuración-del-entorno-y-librerías)  
3. [Carga del dataset de lluvias](#3-carga-del-dataset-de-lluvias)  
4. [Validación de estructura y tipos de datos](#4-validación-de-estructura-y-tipos-de-datos)  
5. [Limpieza de datos](#5-limpieza-de-datos)  
   - [5.1 Conversión de comas decimales y tipos numéricos](#51-conversión-de-comas-decimales-y-tipos-numéricos)  
   - [5.2 Manejo de valores faltantes y registros sin información](#52-manejo-de-valores-faltantes-y-registros-sin-información)  
   - [5.3 Filtrado temporal y espacial (Quito y años de interés)](#53-filtrado-temporal-y-espacial-quito-y-años-de-interés)  
6. [Ingeniería de características base](#6-ingeniería-de-características-base)  
7. [Exportación del dataset limpio](#7-exportación-del-dataset-limpio)  
8. [Resumen del proceso](#8-resumen-del-proceso)  

---

## 1. Contexto y objetivo del ETL

En este notebook se realiza la **limpieza y transformación** del archivo bruto de precipitaciones (INAMHI) para obtener un dataset depurado.

**Objetivo del ETL:** generar un archivo CSV limpio y documentado que sirva como insumo confiable para el notebook principal de análisis, modelado y visualización del riesgo por lluvias en Quito.


## 2. **Configuración del Entorno y Librerías**

In [1]:
# Importar librerías
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")


## 3. **Carga del dataset de lluvias**


In [2]:
# Carga del dataset de lluvias
df_lluvias_raw = pd.read_csv(
    "/content/inamhi-precipitacion-2019diciembre.csv",
    sep=";",
    encoding="latin-1"
)

df_lluvias_raw.head()


,Estacion,NombreEstacion,longitud2,latitud2,altitud,anio,ene,feb,mar,abr,may,jun,jul,ago,sep,oct,nov,dic
0,M0001,INGUINCHO,"-78,408","0,2575",3140,1964,NaN,"81,9",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M0001,INGUINCHO,"-78,408","0,2575",3140,1977,NaN,NaN,NaN,NaN,NaN,61,"1,9","44,1","120,9","179,8","96,2","165,9"
2,M0001,INGUINCHO,"-78,408","0,2575",3140,1978,"98,1","161,5","107,7","196,6","115,6","12,4","37,1","17,6","131,6","62,6","107,5","133,4"
3,M0001,INGUINCHO,"-78,408","0,2575",3140,1979,"71,2","83,2","307,1",139,"252,8","40,5","5,9","104,8","182,1",89,"57,1","27,1"
4,M0001,INGUINCHO,"-78,408","0,2575",3140,1980,"181,9","319,6","101,1","227,6",27,"39,3","2,9","36,8","28,3","137,5","120,6",183


## 4. **Validación de estructura y tipos de datos**

En esta sección se valida:

- Dimensiones del dataset y vista rápida (`shape`, `head`, `info`, `describe`).
- Presencia de las columnas esperadas (`Estacion`, `NombreEstacion`, coordenadas, altitud, año y meses `ene`…`dic`).
- Rangos básicos de `anio`, `latitud2`, `longitud2` y `altitud`.
- Porcentaje de valores faltantes por columna.
- Posibles duplicados por combinación (`Estacion`, `anio`).


In [3]:
# 4. Validación de estructura y tipos de datos

df = df_lluvias_raw  # alias corto para trabajar

# 1) Tamaño del dataset y primeras filas
print("Shape (filas, columnas):", df.shape)
display(df.head())

# 2) Tipos de datos
print("\n=== Info del DataFrame ===")
df.info()

# 3) Estadísticas descriptivas (numéricas y categóricas)
print("\n=== Descripción general (incluye categóricas) ===")
display(df.describe(include="all"))

# 4) Verificar que las columnas clave estén presentes
expected_cols = [
    "Estacion", "NombreEstacion",
    "longitud2", "latitud2", "altitud",
    "anio",
    "ene", "feb", "mar", "abr", "may", "jun",
    "jul", "ago", "sep", "oct", "nov", "dic"
]

missing_cols = [col for col in expected_cols if col not in df.columns]
extra_cols   = [col for col in df.columns if col not in expected_cols]

print("\n=== Columnas esperadas ===")
print("Faltan columnas:", missing_cols if missing_cols else "Ninguna, todas presentes ✅")
print("Columnas extra en el dataset:", extra_cols if extra_cols else "Ninguna")

# 5) Revisar rangos esperados
print("\n=== Rangos de columnas clave ===")
if "anio" in df.columns:
    print("Rango anio:", df["anio"].min(), "→", df["anio"].max())

if "latitud2" in df.columns:
    print("Rango latitud2:", df["latitud2"].min(), "→", df["latitud2"].max())

if "longitud2" in df.columns:
    print("Rango longitud2:", df["longitud2"].min(), "→", df["longitud2"].max())

if "altitud" in df.columns:
    print("Rango altitud:", df["altitud"].min(), "→", df["altitud"].max())

# 6) Revisar valores faltantes
print("\n=== Porcentaje de valores faltantes por columna ===")
na_pct = df.isna().mean().sort_values(ascending=False) * 100
display(na_pct)

# 7) Revisar duplicados por combinación (Estacion, anio)
if {"Estacion", "anio"}.issubset(df.columns):
    dup_count = df.duplicated(subset=["Estacion", "anio"]).sum()
    print(f"\nDuplicados por (Estacion, anio): {dup_count}")
    if dup_count > 0:
        print("Ejemplos de filas duplicadas:")
        display(df[df.duplicated(subset=["Estacion", "anio"], keep=False)].head())
else:
    print("\nNo se puede revisar duplicados (faltan columnas 'Estacion' o 'anio').")


Shape (filas, columnas): (20588, 18)


,Estacion,NombreEstacion,longitud2,latitud2,altitud,anio,ene,feb,mar,abr,may,jun,jul,ago,sep,oct,nov,dic
0,M0001,INGUINCHO,"-78,408","0,2575",3140,1964,NaN,"81,9",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M0001,INGUINCHO,"-78,408","0,2575",3140,1977,NaN,NaN,NaN,NaN,NaN,61,"1,9","44,1","120,9","179,8","96,2","165,9"
2,M0001,INGUINCHO,"-78,408","0,2575",3140,1978,"98,1","161,5","107,7","196,6","115,6","12,4","37,1","17,6","131,6","62,6","107,5","133,4"
3,M0001,INGUINCHO,"-78,408","0,2575",3140,1979,"71,2","83,2","307,1",139,"252,8","40,5","5,9","104,8","182,1",89,"57,1","27,1"
4,M0001,INGUINCHO,"-78,408","0,2575",3140,1980,"181,9","319,6","101,1","227,6",27,"39,3","2,9","36,8","28,3","137,5","120,6",183



=== Info del DataFrame ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20588 entries, 0 to 20587
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Estacion        20588 non-null  object
 1   NombreEstacion  20588 non-null  object
 2   longitud2       20588 non-null  object
 3   latitud2        20588 non-null  object
 4   altitud         20588 non-null  int64 
 5   anio            20588 non-null  int64 
 6   ene             18313 non-null  object
 7   feb             18214 non-null  object
 8   mar             18306 non-null  object
 9   abr             18208 non-null  object
 10  may             18222 non-null  object
 11  jun             18136 non-null  object
 12  jul             18106 non-null  object
 13  ago             18130 non-null  object
 14  sep             18084 non-null  object
 15  oct             18114 non-null  object
 16  nov             18004 non-null  object
 17  dic             18049 

,Estacion,NombreEstacion,longitud2,latitud2,altitud,anio,ene,feb,mar,abr,may,jun,jul,ago,sep,oct,nov,dic
count,20588,20588,20588,20588,20588.000000,20588.000000,18313,18214,18306,18208,18222,18136,18106,18130,18084,18114,18004,18049
unique,947,943,832,843,NaN,NaN,4852,5450,5692,5203,4125,3362,2952,2543,2742,3051,3083,3502
top,M0037,CA¥AR,"-78,5",0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0
freq,60,60,110,93,NaN,NaN,213,126,183,278,748,1693,2859,3396,2339,1765,1838,787
mean,NaN,NaN,NaN,NaN,1470.165242,1989.226928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,1233.827746,15.234333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,0.000000,1960.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,152.000000,1977.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,1453.000000,1988.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,2620.000000,2002.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



=== Columnas esperadas ===
Faltan columnas: Ninguna, todas presentes ✅
Columnas extra en el dataset: Ninguna

=== Rangos de columnas clave ===
Rango anio: 1960 → 2019
Rango latitud2: -0,009755556 → 1,36
Rango longitud2: -75,403 → 0
Rango altitud: 0 → 4200

=== Porcentaje de valores faltantes por columna ===


,0
nov,12.551001
dic,12.332427
sep,12.162425
jul,12.055566
oct,12.016709
ago,11.938994
jun,11.909850
abr,11.560132
feb,11.530989
may,11.492131



Duplicados por (Estacion, anio): 0


## 5. **Limpieza de datos**


## 5.1 **Conversión de comas decimales y tipos numéricos**


## 5.2 **Manejo de valores faltantes y registros sin información**


## 5.3 **Filtrado temporal y espacial (Quito y años de interés)**


## 6. **Ingeniería de características base**


## 7. **Exportación del dataset limpio**


## 8. **Resumen del proceso**